In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [7]:
suffix = 'sim05.csv'
n_samples =500

### BERT-based-uncased

In [8]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7298311444652908
KNN 0.6801125703564728
SVC 0.8030018761726079


In [10]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [11]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [12]:
runAttack(bert_model_wrapper_lr_mr, n_samples, 'attack_results_new/bertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklrsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:27<45:46,  5.55s/it]textattack: Saving checkpoint under "checkpoints\1720171717590.ta.chkpt" at 2024-07-05 17:28:37 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:27<32:48,  3.99s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:37<30:56,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1720171727731.ta.chkpt" at 2024-07-05 17:28:47 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [01:06<35:46,  4.43s/it]textattack: Saving checkpoint under "checkpoints\1720171756231.ta.chkpt" at 2024-07-05 17:29:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 381 / 5 / 99 / 485:  97%|████████████████████▎| 485/500 [20:48<00:38,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1720172938674.ta.chkpt" at 2024-07-05 17:48:58 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 385 / 5 / 100 / 490:  98%|███████████████████▌| 490/500 [21:00<00:25,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1720172950579.ta.chkpt" at 2024-07-05 17:49:10 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 390 / 5 / 100 / 495:  99%|███████████████████▊| 495/500 [21:11<00:12,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1720172961244.ta.chkpt" at 2024-07-05 17:49:21 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 395 / 5 / 100 / 500: 100%|████████████████████| 500/500 [21:25<00:00,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1720172975546.ta.chkpt" at 2024-07-05 17:49:35 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 395 / 5 / 100 / 500: 100%|████████████████████| 500/500 [21:25<00:00,  2.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 395    |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 100    |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 1.0%   |
| Attack success rate:          | 98.75% |
| Average perturbed word %:     | 15.63% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 79.69  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_rf_mr, n_samples, 'attack_results_new/bertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrfsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:19<32:21,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720172995247.ta.chkpt" at 2024-07-05 17:49:55 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:19<23:13,  2.83s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:28<23:33,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1720173004477.ta.chkpt" at 2024-07-05 17:50:04 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 0 / 5 / 15:   3%|▊                        | 15/500 [00:40<21:35,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1720173015683.ta.chkpt" at 2024-07-05 17:50:15 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 337 / 9 / 139 / 485:  97%|███████████████████▍| 485/500 [19:18<00:35,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1720174134621.ta.chkpt" at 2024-07-05 18:08:54 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 341 / 9 / 140 / 490:  98%|███████████████████▌| 490/500 [19:27<00:23,  2.38s/it]textattack: Saving checkpoint under "checkpoints\1720174143279.ta.chkpt" at 2024-07-05 18:09:03 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 344 / 9 / 142 / 495:  99%|███████████████████▊| 495/500 [19:47<00:11,  2.40s/it]textattack: Saving checkpoint under "checkpoints\1720174162666.ta.chkpt" at 2024-07-05 18:09:22 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 348 / 9 / 143 / 500: 100%|████████████████████| 500/500 [19:53<00:00,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1720174169008.ta.chkpt" at 2024-07-05 18:09:29 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 348 / 9 / 143 / 500: 100%|████████████████████| 500/500 [19:53<00:00,  2.39s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 348    |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 143    |
| Original accuracy:            | 71.4%  |
| Accuracy under attack:        | 1.8%   |
| Attack success rate:          | 97.48% |
| Average perturbed word %:     | 14.66% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 79.96  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_knn_mr, n_samples, 'attack_results_new/bertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknnsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:12<20:43,  2.51s/it]textattack: Saving checkpoint under "checkpoints\1720174181732.ta.chkpt" at 2024-07-05 18:09:41 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▎                           | 6/500 [00:12<17:25,  2.12s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:17<14:11,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1720174186544.ta.chkpt" at 2024-07-05 18:09:46 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   3%|▊                         | 15/500 [00:58<31:35,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1720174227787.ta.chkpt" at 2024-07-05 18:10:27 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 332 / 6 / 147 / 485:  97%|███████████████████▍| 485/500 [12:44<00:23,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720174933366.ta.chkpt" at 2024-07-05 18:22:13 after 485 attacks.
[Succeeded / Failed / Skipped / Total] 332 / 6 / 148 / 486:  97%|███████████████████▍| 486/500 [12:44<00:22,  1.57s/it]

[Succeeded / Failed / Skipped / Total] 335 / 6 / 149 / 490:  98%|███████████████████▌| 490/500 [12:55<00:15,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720174944430.ta.chkpt" at 2024-07-05 18:22:24 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 335 / 6 / 150 / 491:  98%|███████████████████▋| 491/500 [12:55<00:14,  1.58s/it]

[Succeeded / Failed / Skipped / Total] 338 / 6 / 151 / 495:  99%|███████████████████▊| 495/500 [13:04<00:07,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720174953518.ta.chkpt" at 2024-07-05 18:22:33 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 341 / 6 / 153 / 500: 100%|████████████████████| 500/500 [13:10<00:00,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720174959964.ta.chkpt" at 2024-07-05 18:22:39 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 341 / 6 / 153 / 500: 100%|████████████████████| 500/500 [13:10<00:00,  1.58s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 341    |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 153    |
| Original accuracy:            | 69.4%  |
| Accuracy under attack:        | 1.2%   |
| Attack success rate:          | 98.27% |
| Average perturbed word %:     | 8.72%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 53.97  |
+-------------------------------+--------+


In [15]:
runAttack(bert_model_wrapper_svc_mr, n_samples, 'attack_results_new/bertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvcsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:27<45:04,  5.46s/it]textattack: Saving checkpoint under "checkpoints\1720174987345.ta.chkpt" at 2024-07-05 18:23:07 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:27<32:14,  3.92s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:39<32:29,  3.98s/it]textattack: Saving checkpoint under "checkpoints\1720174999826.ta.chkpt" at 2024-07-05 18:23:19 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [01:28<47:45,  5.91s/it]textattack: Saving checkpoint under "checkpoints\1720175048657.ta.chkpt" at 2024-07-05 18:24:08 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 381 / 7 / 97 / 485:  97%|████████████████████▎| 485/500 [22:47<00:42,  2.82s/it]textattack: Saving checkpoint under "checkpoints\1720176327608.ta.chkpt" at 2024-07-05 18:45:27 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 385 / 7 / 98 / 490:  98%|████████████████████▌| 490/500 [22:59<00:28,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1720176339349.ta.chkpt" at 2024-07-05 18:45:39 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 390 / 7 / 98 / 495:  99%|████████████████████▊| 495/500 [23:10<00:14,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1720176350958.ta.chkpt" at 2024-07-05 18:45:50 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 395 / 7 / 98 / 500: 100%|█████████████████████| 500/500 [23:24<00:00,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1720176364711.ta.chkpt" at 2024-07-05 18:46:04 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 395 / 7 / 98 / 500: 100%|█████████████████████| 500/500 [23:24<00:00,  2.81s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 395    |
| Number of failed attacks:     | 7      |
| Number of skipped attacks:    | 98     |
| Original accuracy:            | 80.4%  |
| Accuracy under attack:        | 1.4%   |
| Attack success rate:          | 98.26% |
| Average perturbed word %:     | 15.08% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 77.55  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [16]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [18]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [19]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [20]:
runAttack(disbert_model_wrapper_lr_mr, n_samples, 'attack_results_new/disbertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklrsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:23<39:06,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1720176557092.ta.chkpt" at 2024-07-05 18:49:17 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:23<27:56,  3.40s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   2%|▌                         | 10/500 [00:29<24:13,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1720176563057.ta.chkpt" at 2024-07-05 18:49:23 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:44<24:13,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1720176578349.ta.chkpt" at 2024-07-05 18:49:38 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 379 / 4 / 102 / 485:  97%|███████████████████▍| 485/500 [12:41<00:23,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1720177294470.ta.chkpt" at 2024-07-05 19:01:34 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 383 / 4 / 103 / 490:  98%|███████████████████▌| 490/500 [12:49<00:15,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1720177302512.ta.chkpt" at 2024-07-05 19:01:42 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 387 / 4 / 104 / 495:  99%|███████████████████▊| 495/500 [13:00<00:07,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720177313425.ta.chkpt" at 2024-07-05 19:01:53 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 392 / 4 / 104 / 500: 100%|████████████████████| 500/500 [13:14<00:00,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720177327848.ta.chkpt" at 2024-07-05 19:02:07 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 392 / 4 / 104 / 500: 100%|████████████████████| 500/500 [13:14<00:00,  1.59s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 392    |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 104    |
| Original accuracy:            | 79.2%  |
| Accuracy under attack:        | 0.8%   |
| Attack success rate:          | 98.99% |
| Average perturbed word %:     | 15.79% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 77.77  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_rf_mr, n_samples, 'attack_results_new/disbertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrfsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:18<31:10,  3.78s/it]textattack: Saving checkpoint under "checkpoints\1720177346827.ta.chkpt" at 2024-07-05 19:02:26 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▎                           | 6/500 [00:18<26:02,  3.16s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   2%|▌                         | 10/500 [00:30<24:49,  3.04s/it]textattack: Saving checkpoint under "checkpoints\1720177358338.ta.chkpt" at 2024-07-05 19:02:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   3%|▊                        | 15/500 [00:49<26:50,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1720177377726.ta.chkpt" at 2024-07-05 19:02:57 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 349 / 19 / 117 / 485:  97%|██████████████████▍| 485/500 [13:40<00:25,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720178148167.ta.chkpt" at 2024-07-05 19:15:48 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 353 / 19 / 118 / 490:  98%|██████████████████▌| 490/500 [13:49<00:16,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720178157467.ta.chkpt" at 2024-07-05 19:15:57 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 353 / 19 / 119 / 491:  98%|██████████████████▋| 491/500 [13:49<00:15,  1.69s/it]

[Succeeded / Failed / Skipped / Total] 356 / 19 / 120 / 495:  99%|██████████████████▊| 495/500 [13:53<00:08,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1720178161900.ta.chkpt" at 2024-07-05 19:16:01 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 361 / 19 / 120 / 500: 100%|███████████████████| 500/500 [14:04<00:00,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720178172856.ta.chkpt" at 2024-07-05 19:16:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 361 / 19 / 120 / 500: 100%|███████████████████| 500/500 [14:04<00:00,  1.69s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 361   |
| Number of failed attacks:     | 19    |
| Number of skipped attacks:    | 120   |
| Original accuracy:            | 76.0% |
| Accuracy under attack:        | 3.8%  |
| Attack success rate:          | 95.0% |
| Average perturbed word %:     | 16.3% |
| Average num. words per input: | 18.4  |
| Avg num queries:              | 89.19 |
+-------------------------------+-------+


In [22]:
runAttack(disbert_model_wrapper_knn_mr, n_samples, 'attack_results_new/disbertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknnsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:23<37:59,  4.60s/it]textattack: Saving checkpoint under "checkpoints\1720178195963.ta.chkpt" at 2024-07-05 19:16:35 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▎                           | 6/500 [00:23<31:48,  3.86s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   2%|▌                         | 10/500 [00:32<26:49,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1720178205797.ta.chkpt" at 2024-07-05 19:16:45 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   3%|▊                        | 15/500 [00:40<22:05,  2.73s/it]textattack: Saving checkpoint under "checkpoints\1720178213927.ta.chkpt" at 2024-07-05 19:16:53 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 328 / 29 / 128 / 485:  97%|██████████████████▍| 485/500 [13:02<00:24,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1720178955117.ta.chkpt" at 2024-07-05 19:29:15 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 332 / 29 / 129 / 490:  98%|██████████████████▌| 490/500 [13:07<00:16,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1720178960141.ta.chkpt" at 2024-07-05 19:29:20 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 335 / 29 / 131 / 495:  99%|██████████████████▊| 495/500 [13:18<00:08,  1.61s/it]textattack: Saving checkpoint under "checkpoints\1720178971919.ta.chkpt" at 2024-07-05 19:29:31 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 335 / 29 / 132 / 496:  99%|██████████████████▊| 496/500 [13:19<00:06,  1.61s/it]

[Succeeded / Failed / Skipped / Total] 337 / 31 / 132 / 500: 100%|███████████████████| 500/500 [13:34<00:00,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1720178987567.ta.chkpt" at 2024-07-05 19:29:47 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 337 / 31 / 132 / 500: 100%|███████████████████| 500/500 [13:34<00:00,  1.63s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 337    |
| Number of failed attacks:     | 31     |
| Number of skipped attacks:    | 132    |
| Original accuracy:            | 73.6%  |
| Accuracy under attack:        | 6.2%   |
| Attack success rate:          | 91.58% |
| Average perturbed word %:     | 11.09% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 82.93  |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_svc_mr, n_samples, 'attack_results_new/disbertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvcsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:17<29:41,  3.60s/it]textattack: Saving checkpoint under "checkpoints\1720179005662.ta.chkpt" at 2024-07-05 19:30:05 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:18<21:15,  2.59s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   2%|▌                         | 10/500 [00:23<19:14,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1720179011239.ta.chkpt" at 2024-07-05 19:30:11 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:38<20:33,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1720179025829.ta.chkpt" at 2024-07-05 19:30:25 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 377 / 4 / 104 / 485:  97%|███████████████████▍| 485/500 [12:03<00:22,  1.49s/it]textattack: Saving checkpoint under "checkpoints\1720179711516.ta.chkpt" at 2024-07-05 19:41:51 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 380 / 4 / 106 / 490:  98%|███████████████████▌| 490/500 [12:09<00:14,  1.49s/it]textattack: Saving checkpoint under "checkpoints\1720179717532.ta.chkpt" at 2024-07-05 19:41:57 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 380 / 4 / 107 / 491:  98%|███████████████████▋| 491/500 [12:09<00:13,  1.49s/it]

[Succeeded / Failed / Skipped / Total] 384 / 4 / 107 / 495:  99%|███████████████████▊| 495/500 [12:17<00:07,  1.49s/it]textattack: Saving checkpoint under "checkpoints\1720179724854.ta.chkpt" at 2024-07-05 19:42:04 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 389 / 4 / 107 / 500: 100%|████████████████████| 500/500 [12:29<00:00,  1.50s/it]textattack: Saving checkpoint under "checkpoints\1720179736726.ta.chkpt" at 2024-07-05 19:42:16 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 389 / 4 / 107 / 500: 100%|████████████████████| 500/500 [12:29<00:00,  1.50s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 389    |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 107    |
| Original accuracy:            | 78.6%  |
| Accuracy under attack:        | 0.8%   |
| Attack success rate:          | 98.98% |
| Average perturbed word %:     | 15.64% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 77.61  |
+-------------------------------+--------+


### CLIP

In [24]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [26]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [27]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [28]:
runAttack(clip_model_wrapper_lr_mr, n_samples, 'attack_results_new/clipmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklrsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:20<34:17,  4.16s/it]textattack: Saving checkpoint under "checkpoints\1720180188615.ta.chkpt" at 2024-07-05 19:49:48 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▎                           | 6/500 [00:20<28:42,  3.49s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   2%|▌                         | 10/500 [00:28<23:02,  2.82s/it]textattack: Saving checkpoint under "checkpoints\1720180196044.ta.chkpt" at 2024-07-05 19:49:56 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:   3%|▊                        | 15/500 [00:46<25:14,  3.12s/it]textattack: Saving checkpoint under "checkpoints\1720180214669.ta.chkpt" at 2024-07-05 19:50:14 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 356 / 2 / 127 / 485:  97%|███████████████████▍| 485/500 [09:29<00:17,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720180737189.ta.chkpt" at 2024-07-05 19:58:57 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 359 / 2 / 129 / 490:  98%|███████████████████▌| 490/500 [09:34<00:11,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720180742587.ta.chkpt" at 2024-07-05 19:59:02 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 359 / 2 / 130 / 491:  98%|███████████████████▋| 491/500 [09:34<00:10,  1.17s/it]

[Succeeded / Failed / Skipped / Total] 362 / 2 / 131 / 495:  99%|███████████████████▊| 495/500 [09:37<00:05,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720180745148.ta.chkpt" at 2024-07-05 19:59:05 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 367 / 2 / 131 / 500: 100%|████████████████████| 500/500 [09:46<00:00,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720180754387.ta.chkpt" at 2024-07-05 19:59:14 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 367 / 2 / 131 / 500: 100%|████████████████████| 500/500 [09:46<00:00,  1.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 367    |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 131    |
| Original accuracy:            | 73.8%  |
| Accuracy under attack:        | 0.4%   |
| Attack success rate:          | 99.46% |
| Average perturbed word %:     | 13.73% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 66.12  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_rf_mr, n_samples, 'attack_results_new/clipmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrfsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:15<25:00,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1720180769631.ta.chkpt" at 2024-07-05 19:59:29 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▍                           | 7/500 [00:15<17:54,  2.18s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   2%|▌                         | 10/500 [00:24<19:48,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1720180778739.ta.chkpt" at 2024-07-05 19:59:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   3%|▊                        | 15/500 [00:34<18:48,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1720180789395.ta.chkpt" at 2024-07-05 19:59:49 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 327 / 14 / 144 / 485:  97%|██████████████████▍| 485/500 [10:59<00:20,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720181414125.ta.chkpt" at 2024-07-05 20:10:14 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 331 / 14 / 145 / 490:  98%|██████████████████▌| 490/500 [11:05<00:13,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720181419543.ta.chkpt" at 2024-07-05 20:10:19 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 331 / 14 / 146 / 491:  98%|██████████████████▋| 491/500 [11:05<00:12,  1.35s/it]

[Succeeded / Failed / Skipped / Total] 333 / 14 / 148 / 495:  99%|██████████████████▊| 495/500 [11:07<00:06,  1.35s/it]textattack: Saving checkpoint under "checkpoints\1720181422035.ta.chkpt" at 2024-07-05 20:10:22 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 333 / 14 / 149 / 496:  99%|██████████████████▊| 496/500 [11:07<00:05,  1.35s/it]

[Succeeded / Failed / Skipped / Total] 337 / 14 / 149 / 500: 100%|███████████████████| 500/500 [11:12<00:00,  1.35s/it]textattack: Saving checkpoint under "checkpoints\1720181427078.ta.chkpt" at 2024-07-05 20:10:27 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 337 / 14 / 149 / 500: 100%|███████████████████| 500/500 [11:12<00:00,  1.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 337    |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 149    |
| Original accuracy:            | 70.2%  |
| Accuracy under attack:        | 2.8%   |
| Attack success rate:          | 96.01% |
| Average perturbed word %:     | 13.47% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 76.29  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_knn_mr, n_samples, 'attack_results_new/clipmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknnsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   1%|▎                           | 5/500 [00:26<43:02,  5.22s/it]textattack: Saving checkpoint under "checkpoints\1720181453244.ta.chkpt" at 2024-07-05 20:10:53 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   2%|▌                         | 10/500 [00:29<24:07,  2.95s/it]textattack: Saving checkpoint under "checkpoints\1720181456694.ta.chkpt" at 2024-07-05 20:10:56 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   3%|▊                        | 15/500 [00:41<22:34,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1720181469041.ta.chkpt" at 2024-07-05 20:11:09 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 310 / 23 / 152 / 485:  97%|██████████████████▍| 485/500 [11:08<00:20,  1.38s/it]textattack: Saving checkpoint under "checkpoints\1720182095445.ta.chkpt" at 2024-07-05 20:21:35 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 314 / 23 / 153 / 490:  98%|██████████████████▌| 490/500 [11:13<00:13,  1.38s/it]textattack: Saving checkpoint under "checkpoints\1720182101149.ta.chkpt" at 2024-07-05 20:21:41 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 314 / 23 / 154 / 491:  98%|██████████████████▋| 491/500 [11:14<00:12,  1.37s/it]

[Succeeded / Failed / Skipped / Total] 316 / 23 / 156 / 495:  99%|██████████████████▊| 495/500 [11:16<00:06,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720182103868.ta.chkpt" at 2024-07-05 20:21:43 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 316 / 23 / 158 / 497:  99%|██████████████████▉| 497/500 [11:16<00:04,  1.36s/it]

[Succeeded / Failed / Skipped / Total] 319 / 23 / 158 / 500: 100%|███████████████████| 500/500 [11:20<00:00,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720182107931.ta.chkpt" at 2024-07-05 20:21:47 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 319 / 23 / 158 / 500: 100%|███████████████████| 500/500 [11:20<00:00,  1.36s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 319    |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 158    |
| Original accuracy:            | 68.4%  |
| Accuracy under attack:        | 4.6%   |
| Attack success rate:          | 93.27% |
| Average perturbed word %:     | 10.3%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 73.2   |
+-------------------------------+--------+


In [31]:
runAttack(clip_model_wrapper_svc_mr, n_samples, 'attack_results_new/clipmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvcsim05.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:14<23:48,  2.89s/it]textattack: Saving checkpoint under "checkpoints\1720182122452.ta.chkpt" at 2024-07-05 20:22:02 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▍                           | 7/500 [00:14<17:02,  2.07s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   2%|▌                         | 10/500 [00:19<15:46,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1720182127343.ta.chkpt" at 2024-07-05 20:22:07 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   3%|▊                        | 15/500 [00:33<18:04,  2.24s/it]textattack: Saving checkpoint under "checkpoints\1720182141563.ta.chkpt" at 2024-07-05 20:22:21 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 358 / 2 / 125 / 485:  97%|███████████████████▍| 485/500 [09:43<00:18,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720182691506.ta.chkpt" at 2024-07-05 20:31:31 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 360 / 2 / 128 / 490:  98%|███████████████████▌| 490/500 [09:47<00:11,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720182695397.ta.chkpt" at 2024-07-05 20:31:35 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 364 / 2 / 129 / 495:  99%|███████████████████▊| 495/500 [09:51<00:05,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720182699226.ta.chkpt" at 2024-07-05 20:31:39 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 369 / 2 / 129 / 500: 100%|████████████████████| 500/500 [10:00<00:00,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720182709023.ta.chkpt" at 2024-07-05 20:31:49 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 369 / 2 / 129 / 500: 100%|████████████████████| 500/500 [10:01<00:00,  1.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 369    |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 129    |
| Original accuracy:            | 74.2%  |
| Accuracy under attack:        | 0.4%   |
| Attack success rate:          | 99.46% |
| Average perturbed word %:     | 13.57% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 66.56  |
+-------------------------------+--------+
